# DISEÑO DE  VIGAS DOBLEMENTE REFORZADAS | DOUBLE REINFORCED CONCRETE BEAM DESIGN- CLEISON MANRIQUE
Este material está sujeto a los términos y condiciones de la licencia [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Se permite el uso gratuito para cualquier propósito no comercial.
Autor: Cleison Armando Manrique Aguirre

## 1. Ecuaciones de equilibrio 

Ecuacion que rige el comportamiento de una viga cuando ...  $\epsilon'>\epsilon_y$ ... el acero empieza a fluir.

$$ A_s \cdot f_y = 0.85 \cdot f'c \cdot \beta_1 \cdot c \cdot b + A'_s \cdot fy $$

Ecuacion que rige el comportamiento de una viga cuando ...  $\epsilon'<\epsilon_y$ ... el acero empieza a fluir.

$$ A_s \cdot f_y = 0.85 \cdot f'c \cdot \beta_1 \cdot c \cdot b + A'_s \cdot \frac{c-d'}{c} \cdot 0.003 \cdot E_s$$

Primera iteracion se considera la suposicion de la ecuacion 1

In [13]:
import numpy as np
 
#tol=1.0e-8

def biseccion(f, a, b, tol):
    
    if a > b:
        raise ValueError("Intervalo mal definido.")
    if tol <= 0:
        raise ValueError("La cota de error debe ser un número positivo.")
        
    x = (a + b) / 2.0
    i=1
    while True:
        if b - a < tol:
            i+=1
            return(x,i)
        # Utilizamos la función signo para evitar errores de precisión
        elif np.sign(f(a)) * np.sign(f(x)) > 0: #+*- o +*+
            i+=1 #para el contador
            a = x
        else:
            i+=1
            b = x
        x = (a + b) / 2.0
    return(x,i)


## 2. Iteracion

In [14]:
def iteracion(fc,fy,Es,d,b,dp,A_s,A_sp):
    #variables
    global fc1,fy1,Es1,d1,b1,dp1,A_s1,A_sp1
    fc1=210     #esfuerzo de compresion concreto
    fy1=4200    #esferzo de fluencia del acero 
    Es1=2039000 #Modulo de elasticidad acero
    d1=41.5        #peralte h-r
    b1=30        #ancho
    dp1=6.5      #recubrimiento superior
    A_s1=30.6    #acero inferior
    A_sp1=12.2      #acero superior
        
    # valor betha del rectangulo de witney
    if (fc <= 280):
        beta = 0.85
    elif (280 < fc & fc < 360):
        beta = 0.85-(0.05*(fc-360))/100
    else:
        beta = 0.65 
        
    c=((A_s)-(A_sp))*fy/(0.85*fc*b*beta)    
    epsilon_sp=(c-dp)*0.003/c 
    epsilon_y=fy/Es
    
    #si el acero inferior fluye
    if (epsilon_sp>=epsilon_y):
        #print("El acero inferior fluye")
        epsilon_t=(d-c)*0.003/c
        
        #el acero superior fluye?
        if (epsilon_t >= 0.004):
            print("El acero superior es Ductil")
        else :
            print("El acero superior es Fragil")
        a=beta*c
        
        #valor phi de reduccion de resistencia del acero 
        phi=0.65+((epsilon_t)-(epsilon_y))*0.25/(0.005-(epsilon_y))
        if (phi < 0.65):
            phi=0.65
        elif (phi <= 0.9) & (0.65 <= phi):
            phi=phi 
        else :
            phi=0.9
        
        #valor del esfuerzo si fluye
        f_sp=fy             
        print("El acero inferior fluye")
        A_s2=A_sp*f_sp/fy
        A_s1=A_s-A_s2
        Mu=((phi)*((A_s1)*fy*(d-a/2)))+((A_sp)*(f_sp)*(d-dp)) 
        print("El momento resistente ultimo es",Mu,c,a,beta,"kg-cm")
    else :
        print("El acero NO fluye") 
    q_1=round(0.85*fc*beta*b1,3)
    w_1=round(A_sp1*0.003*Es1-A_s1*fy1,3)
    e_1=round(-A_sp1*dp1*0.003*Es1,3)
    print("Los parametros de la ecuacion cuadratica son a,b,c como sigue a continuacion",q_1,w_1,e_1)

In [15]:
iteracion(210,4200,2100000,41.5,30,6.5,30.6,12.2)

El acero NO fluye
Los parametros de la ecuacion cuadratica son a,b,c como sigue a continuacion 4551.75 -53892.6 -485078.1


**IMPORTANTE| FLUYE O NO FLUYE EL ACERO INFERIOR**

In [16]:
iteracion(fc1,fy1,Es1,d1,b1,dp1,A_s1,A_sp1)

El acero NO fluye
Los parametros de la ecuacion cuadratica son a,b,c como sigue a continuacion 4551.75 -53892.6 -485078.1


**Si el acero no fluye**

In [17]:
tol=1.0e-8
def no_fluye(x):
    #q_1=0.85*fc*beta*b1
    #w_1=A_sp1*0.003*Es1-A_s1*fy1
    #e_1=-A_sp1*dp1*0.003*Es1
    q=4551.75
    w=-53892.6
    e=-485078.1
    return(q*x**2+w*x+e)
print(biseccion(no_fluye,1,20,tol))
print(no_fluye(biseccion(no_fluye,1,20,tol)[0]))

(17.820231653517112, 33)
0.0003797975368797779


In [21]:
def momento_no_fluye(c1):
    if (fc1 <= 4000):
        beta = 0.85
    elif (4000 < fc1 & fc1 < 8000):
        beta = 0.85-(0.05*(fc1-4000))/1000
    else:
        beta = 0.65 
    epsilon_y=fy1/Es1
    epsilon_sp=(c1-dp1)*0.003/c1
    f_sp=epsilon_sp*Es1
    a=beta*c1
    A_s2=A_sp1*f_sp/fy1
    A_s11=A_s1-A_s2
    epsilon_t=(d1-c1)*0.003/c1
    phi=0.65+((epsilon_t)-(epsilon_y))*0.25/(0.005-(epsilon_y))
    if (phi < 0.65):
        phi=0.65
    elif (phi <= 0.9) & (0.65 <= phi):
        phi=phi 
    else :
        phi=0.9
    Mu=((phi)*((A_s11)*fy1*(d1-a/2)))+((A_sp1)*(f_sp)*(d1-dp1)) 
    print("El momento resistente ultimo es",Mu,"kg-cm")

In [22]:
momento_no_fluye(17.8)

El momento resistente ultimo es 3900964.55748313 kg-cm
